In [1]:
from mxnet import nd, autograd
from mxnet.gluon import nn

def corr2d(X, K):
    n, m = K.shape
    Y = nd.zeros((X.shape[0]-n+1, X.shape[1]-m+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+n, j:j+m]*K).sum()
    return Y

D:\Programdata\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
D:\Programdata\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:

def corr2d_multi_in(X, K):
    # 我们首先沿着 X 和 K 的第 0 维（通道维）遍历。然后使用 * 将结果列表 (list) 变成
    # add_n 的位置参数（positional argument）来进行相加。
    return nd.add_n(*[corr2d(x, k) for x, k in zip(X, K)])



In [3]:

X = nd.array([[[0,1,2], [3,4,5], [6,7,8]],
              [[1,2,3], [4,5,6], [7,8,9]]])
K = nd.array([[[0,1], [2,3]], [[1,2], [3,4]]])

corr2d_multi_in(X, K)




[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

In [4]:
def corr2d_multi_in_out(X, K):
    # 对 K 的第 0 维遍历，每次同输入 X 做相关计算。所有结果使用 nd.stack 合并在一起。
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [5]:
K = nd.stack(K, K+1, K+2)
K.shape

(3, 2, 2, 2)

In [6]:
corr2d_multi_in_out(X, K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X)
    return Y.reshape((c_o, h, w))

In [8]:
X = nd.random.uniform(shape=(3,3,3))
K = nd.random.uniform(shape=(2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
(Y1-Y2).norm().asscalar() < 1e-6

True

- 假设输入大小为ci×h×w，我们使用co×ci×kh×kw的核，而且使用(ph,pw)填充和(sh,sw) 步幅，那么这个卷积层的前向计算需要多少次乘法，多少次加法？
- 翻倍输入通道ci和输出通道co会增加多少倍计算？翻倍填充呢？
- 如果使用kh=kw=1，能减低多少倍计算？
- Y1和Y2结果完全一致吗？原因是什么？
- 对于非1×1卷积层，如果将其也表示成一个矩阵乘法。